In [38]:
import open3d as o3d
import numpy as np
import math

In [39]:
pcd = o3d.io.read_point_cloud("data/test_capture.ply")
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd])

PointCloud with 72198 points.
[[-0.44946289  0.39306641 -0.94238281]
 [-0.4453125   0.39233398 -0.94042969]
 [-0.44165039  0.3918457  -0.93945312]
 ...
 [ 0.45117188 -0.33422852 -0.85693359]
 [ 0.45385742 -0.33422852 -0.85693359]
 [ 0.45678711 -0.33422852 -0.85693359]]


In [40]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)

[a, b, c, d] = plane_model
pcd = pcd.translate((0,0,d/c))
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0, 0, 1.0])

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [41]:
cos_theta = c / math.sqrt(a**2 + b**2 + c**2)
sin_theta = math.sqrt((a**2+b**2)/(a**2 + b**2 + c**2))
u_1 = b / math.sqrt(a**2 + b**2 )
u_2 = -a / math.sqrt(a**2 + b**2)

rotation_matrix = np.array([[cos_theta + u_1**2 * (1-cos_theta), u_1*u_2*(1-cos_theta), u_2*sin_theta],
                            [u_1*u_2*(1-cos_theta), cos_theta + u_2**2*(1- cos_theta), -u_1*sin_theta],
                            [-u_2*sin_theta, u_1*sin_theta, cos_theta]])

print(rotation_matrix)

[[ 0.99256449 -0.00261571  0.12169176]
 [-0.00261571  0.99907983  0.04280959]
 [-0.12169176 -0.04280959  0.99164432]]


In [42]:
pcd.rotate(rotation_matrix)
o3d.visualization.draw_geometries([pcd, axes])

In [43]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)

[a, b, c, d] = plane_model
pcd = pcd.translate((0,0,d/c))
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([outlier_cloud])

In [44]:
cl, ind = outlier_cloud.remove_statistical_outlier(nb_neighbors=30,
                                                    std_ratio=2.0)
stockpile_pcd = outlier_cloud.select_by_index(ind)
o3d.visualization.draw_geometries([stockpile_pcd])


In [48]:
hull, _ = stockpile_pcd.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([hull])

In [19]:
hull.get_volume()

0.000712402232093026

In [34]:
bounding_box = stockpile_pcd.get_oriented_bounding_box()
o3d.visualization.draw_geometries([stockpile_pcd, bounding_box])
print(bounding_box)

OrientedBoundingBox: center: (-0.063219, -0.0726969, 0.0581613), extent: 0.518175, 0.3296230.120243)


In [35]:
bounding_box.volume()

0.020537804903675454

In [46]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    stockpile_pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([rec_mesh])